## 🛰️ Imágenes de Cobertura y Uso del Suelo de MapBiomas Colombia – Colección 2 (1985–2023)

La Colección 2 de MapBiomas Colombia (1985–2023) ofrece mapas anuales de cobertura y uso del suelo generados a partir de series temporales de imágenes satelitales. Estos datos están disponibles de forma pública como un **asset en Google Earth Engine (GEE)**.

### 📦 Asset disponible en GEE
- **asset:** projects/mapbiomas-public/assets/colombia/collection2/mapbiomas_colombia_collection2_integration_v1

Este asset corresponde a una **imagen multibanda**, donde cada banda representa un año específico. Por ejemplo:

- `classification_1985`
- `classification_2001`
- `classification_2022`
- `classification_2023`

---

## 🧪 Requisitos para usar en JupyterLab desde Docker

### 1. Instalar librerías necesarias dentro del contenedor

En una celda de Jupyter o desde la terminal del contenedor, es necesario ejecutar:

```bash
conda install conda-forge::earthengine-api
conda install conda-forge::geemap
conda install conda-forge::ipyleaflet
```

| 📦 Paquete           | 📝 Función                                                                 |
|----------------------|---------------------------------------------------------------------------|
| `earthengine-api`    | Para acceder al catálogo de Google Earth Engine (GEE)                     |
| `geemap`             | Para visualizar datos geoespaciales en mapas interactivos                |
| `jupyter-leaflet`    | Para mostrar mapas dentro de JupyterLab usando `geemap`                  |

### 2. 🔐 Autenticación en Google Earth Engine

Desde el notebook, ejecuta el siguiente código para autenticar tu cuenta de Google Earth Engine:

```python
import ee

# Solo es necesario la primera vez
ee.Authenticate()
ee.Initialize()
```
Esto abrirá un enlace en tu navegador donde deberás iniciar sesión con tu cuenta de Google asociada a Earth Engine.
Luego, copia el código de verificación que te aparece y pégalo en el campo solicitado del notebook.

✅ Este paso solo es necesario una vez por entorno. Si cierras el contenedor o cambias de entorno, puede que debas volver a autenticarte.


In [1]:
import sys
sys.path.append("../src")
import data_preprocessing
data_preprocessing.authenticate_earth_engine()

✅Auteticación y conexión con Google Earth Engine exitosa


In [2]:
imagen = data_preprocessing.load_mapbiomas_asset()
print (imagen.bandNames().getInfo())

['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_2020', 'classification_2021', 'classification_2022', 'classification_2023']


In [3]:
# Obtener el sistema de coordenadas de la imagen
projection = imagen.projection()

# Mostrar el sistema de coordenadas
print(projection.getInfo())

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.0002694945852358564, 0, -82.8156860429787, 0, -0.0002694945852358564, 16.131676377633134]}


# ✂️ Recorte y reproyección de capas vectoriales (SHP) a EPSG:
Este flujo permite recortar y reproyectar capas como departamentos, resguardos indígenas y áreas protegidas.

## 1. 📁 Cargar los archivos .shp

In [4]:
dept, reg, resg, runap = data_preprocessing.load_geospatial_layers("DATOS")

✅Finalizó la función load_geospatial_layers


## 2. 🧭 Asegurar proyecciones (planas) y recortar por Colombia

In [5]:
# Reproyectar las capas
dept_9377, resg_9377, runap_9377, reg_9377 = data_preprocessing.reproject_layers_pl(dept, resg, runap, reg)

✅Finalizó la función reproject_layers


## 3 🧭 Selección del departamento de la amazonía con menor area de: Parques Nacionales Naturales y Resguardos indígenas

Apartir de la exploración de los datos se buscar identificar el departemento que pueda presentar mayor transformación del uso del suelo, por lo cual, se busca calcular la proporción de área de parques naturales y resguardos indígenas en departamentos de la región amazónica de Colombia utilizando operaciones espaciales con GeoPandas. 

-1. Se determina qué departamentos tienen al menos un 30% de superposición con la región amazónica. 
-2. Se calcula las áreas ocupadas por zonas protegidas (RUNAP) y resguardos (resguardos_9377) dentro de esos departamentos.
-3. Se estima el área libre restante y se determina cuál departamento tiene mayor superficie sin protección.

In [6]:
#Seleccionar departamento
dptos_area, resguardos_area, parques_area, dptos_amazonicos = data_preprocessing.dep_con_menor_area_protegida (dept_9377, resg_9377, runap_9377)

/notebooks/DEFORESTACION/PROJECT_TRANSITION/../src/data_preprocessing.py:98: UserWarning: `keep_geom_type=True` in overlay resulted in 227 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  interseccion = gpd.overlay(dept_9377, resg_9377, how='intersection')


Departamentos amazónicos seleccionados:
   DeCodigo    DeNombre         DeArea  \
12       95    Guaviare   55403.201009   
21       44  La Guajira   20572.746968   
28       91    Amazonas  107982.504784   
29       94     Guainía   70918.056082   
30       97      Vaupés   53060.617565   
31       99     Vichada   99094.730464   

                                   DeNorma    SHAPE_Leng    SHAPE_Area  \
12  Constitución Política de Colombia 1991  2.150053e+06  5.540320e+10   
21                      Ley No. 19 de 1964  1.186669e+06  2.057275e+10   
28  Constitución Política de Colombia 1991  2.795295e+06  1.079825e+11   
29  Constitución Política de Colombia 1991  2.375999e+06  7.091806e+10   
30  Constitución Política de Colombia 1991  2.247336e+06  5.306062e+10   
31  Constitución Política de Colombia 1991  1.952924e+06  9.909473e+10   

                                             geometry    area_total  
12  POLYGON ((4926985.806 1811520.488, 4926985.806...  5.540320e+10  
21  PO

/notebooks/DEFORESTACION/PROJECT_TRANSITION/../src/data_preprocessing.py:118: UserWarning: `keep_geom_type=True` in overlay resulted in 6 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  resguardos_inter = gpd.overlay(resg_9377, dptos_amazonicos, how='intersection')


## 4. ✂️ Recortar capas al área de estudio

In [8]:
caqueta, resguardos_caqueta, parques_caqueta = data_preprocessing.recort_capas(dept_9377, resg_9377, runap_9377)

Cantidad de parques en Caquetá: 99
Cantidad de resguardos en Caquetá: 54


## 5.REPROYECCIÓN DE CAPAS 4326

In [9]:
caqueta_4326, resguardos_caqueta_4326, parques_caqueta_4326 = data_preprocessing.reproject_layers(caqueta, resguardos_caqueta, parques_caqueta)

✅Finalizó la función reproject_layers


## 6. ✅ Guardar las capas recortadas y reproyectadas

In [10]:
data_preprocessing.save_layers(caqueta_4326, resguardos_caqueta_4326, parques_caqueta_4326)

✅ Capas guardadas correctamente en: CAPAS_SALIDAS


## 7. ✅ Recortar raster al área de estudio

In [11]:
cober_clipped = data_preprocessing.clip_raster_to_region(caqueta_4326)

✅ Capas  convertidas correctamente
